In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [ ]:
custom_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomRotation(degrees=45),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=custom_transform,
)

train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)

val_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ]),
)

val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.fc1 = nn.Linear(512 * 14 * 14, 1024)
        self.bn5 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn6 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 102)
        self.dropout = nn.Dropout(0.5)  # Increase dropout rate

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 512 * 14 * 14)

        x = F.relu(self.bn5(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn6(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

model = NeuralNetwork().to(device)


In [ ]:
model = NeuralNetwork().to(device)


loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)


In [ ]:
# Training loop
def train_loop(dataloader, model, loss_fn, optimizer, device):
    model.train()
    running_loss = 0.0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Validation loop
def val_loop(dataloader, model, loss_fn, device):
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    return val_loss, correct * 100


In [ ]:
# Training
start = time.time()
epochs = 50
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer, device)
    val_loss, val_accuracy = val_loop(val_dataloader, model, loss_fn, device)

    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}")

    scheduler.step(val_loss)  # Adjust learning rate based on validation loss

end = time.time()
print(f"Training took: {(end - start) / 60:.2f} minutes!")

Epoch 1/50
Train Loss: 4.7405, Val Loss: 4.6109, Val Accuracy: 1.67
Epoch 2/50
Train Loss: 4.6172, Val Loss: 4.4364, Val Accuracy: 4.90
Epoch 3/50
Train Loss: 4.5111, Val Loss: 4.2930, Val Accuracy: 8.04
Epoch 4/50
Train Loss: 4.4497, Val Loss: 4.1904, Val Accuracy: 10.29
Epoch 5/50
Train Loss: 4.3838, Val Loss: 4.1307, Val Accuracy: 13.04
Epoch 6/50
Train Loss: 4.3257, Val Loss: 4.0310, Val Accuracy: 12.55
Epoch 7/50
Train Loss: 4.2605, Val Loss: 3.9432, Val Accuracy: 14.02
Epoch 8/50
Train Loss: 4.2235, Val Loss: 3.9159, Val Accuracy: 14.22
Epoch 9/50
Train Loss: 4.1994, Val Loss: 3.8892, Val Accuracy: 16.57
Epoch 10/50
Train Loss: 4.1408, Val Loss: 3.8679, Val Accuracy: 15.49
Epoch 11/50
Train Loss: 4.0829, Val Loss: 3.7717, Val Accuracy: 17.16
Epoch 12/50
Train Loss: 4.1084, Val Loss: 3.7447, Val Accuracy: 18.92
Epoch 13/50
Train Loss: 4.0661, Val Loss: 3.6788, Val Accuracy: 17.65
Epoch 14/50
Train Loss: 3.9919, Val Loss: 3.6671, Val Accuracy: 18.63
Epoch 15/50
Train Loss: 3.9776, 